In [ ]:
## Задание 1
### Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('ratings.csv')
df.head()

In [ ]:
# Подсчитаем число оценок
df['userId'].value_counts()

In [ ]:
# Сформируем фильтр из более 100 оценок пользователя
df['userId'].value_counts() > 100

In [ ]:
# Получим значения userId, для которых количество оценок более 100
df['userId'].value_counts()[df['userId'].value_counts() > 100]

In [ ]:
# Выведем все строки userID и timestamp, где у пользователя более 100 оценок
filtered = df[df['userId'].isin(df['userId'].value_counts()[
    df['userId'].value_counts() > 100].index)][['userId', 'timestamp']]
filtered

In [ ]:
# Выведем разницу максимального и минимального значения timestamp
filtered.groupby('userId')['timestamp'].agg(np.ptp)

In [ ]:
# Среднее значение жизни пользователей, выставивших более 100 оценок
filtered.groupby('userId')['timestamp'].agg(np.ptp).mean()

In [ ]:
## Задание 2
### Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
### - таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
### - аналогичную таблицу по типам выручки с указанием адреса клиента
### Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
# Входные данные
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
client_base.client_id.to_frame().merge(rzd, how='outer').merge(
    auto, how='outer').merge(air, how='outer').fillna(0)

In [ ]:
client_base.merge(rzd, how='outer').merge(
    auto, how='outer').merge(air, how='outer').fillna(0)

In [ ]:
## Задание 3
### В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т.е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

### - У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуются одно-два числа. Как их получить?
### - Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчета пункта 1?
### - Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [ ]:
# Для добавления координат в таблицу визитов и покупок потребуются уникальные идентификаторы посещения магазина и пробитого чека, чтобы иметь возможность различать входы из разных мест и факты покупок/их отсутствие. Можно связать покупки и посещения отдельной таблицей, чтобы избежать использование лишней памяти на поля NaN при отсутствии покупок.

In [ ]:
coordinates = pd.DataFrame(
    {
        'user_id': [1, 2, 3, 4, 5],
        'coordinates': ['36.123456 57.123456', '36.123666 57.123456', '36.123456 57.123673', '36.123456 57.456123', '36.456123 57.123789'],
        'visit_id': [101, 301, 305, 896, 896],
        'bill_id': [np.nan, 6801, 2312, np.nan, np.nan],
    }
)

coordinates

In [ ]:
# Из координат можно получить данные об активности пользователя и его предпочтениях в данной области, сферу интересов на основе соответствия координат и находящихся по ним объектов.